# AmazonレビューのClassification

In [1]:
import pandas as pd
import math
import numpy as np

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, Dropout, Bidirectional
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.callbacks import EarlyStopping

# まずはデータを準備

In [2]:
# データをロード
files = {"train": "amazon_reviews_train.csv", "test": "amazon_reviews_test.csv"}
train_df = pd.read_csv(files["train"], index_col=0)
test_df  = pd.read_csv(files["test"], index_col=0)

In [3]:
# MeCab で分かち書き
import MeCab
from tqdm import tqdm
tqdm.pandas()
def yield_wakati_terms(text):
    mecab = MeCab.Tagger ("-Owakati")
    wakati_text = mecab.parse (text)
    return wakati_text.strip()
        
train_df["terms"] = train_df["sentence1"].progress_map(yield_wakati_terms)
test_df["terms"]  = test_df["sentence1"].progress_map(yield_wakati_terms)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1961/1961 [00:03<00:00, 499.83it/s]


# TF-IDF＋Naive Bayesで分類する

In [4]:
# TF-IDFを構築する
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(use_idf=True, token_pattern=u'(?u)\\b\\w+\\b', ngram_range=(1, 2))
vectorizer.fit(train_df["terms"].values)

TfidfVectorizer(ngram_range=(1, 2), token_pattern='(?u)\\b\\w+\\b')

In [5]:
# 必要なデータを抽出
X_train = vectorizer.transform(train_df["terms"].values)
X_test  = vectorizer.transform(test_df["terms"].values)
y_train = train_df["label"].astype(float)
y_test  = test_df["label"].astype(float)
num_labels = len(y_train.unique())

In [25]:
# モデル構築とtrainingを行う
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score

parameters = [
    {'alpha': [1, 0.1, 0.01, 0.001, 0.0001, 0.00001, 0.000001]}
    ]

clf = GridSearchCV(
    MultinomialNB(),
    parameters,
    cv=5,
    scoring={"acc": "accuracy", "logloss": "neg_log_loss"}, 
    refit="logloss",
    verbose=0)
clf.fit(X_train, y_train)
print("best params:", clf.best_params_)
print("best score:", clf.best_score_)

y_pred_proba = clf.best_estimator_.predict_proba(X_test)
y_pred = clf.best_estimator_.predict(X_test)
print("validation log_loss:", log_loss(y_test, y_pred_proba))
print("validation accuracy:", accuracy_score(y_test, y_pred))

best params: {'alpha': 0.01}
best score: -0.6757729264785761
validation log_loss: 0.5519795891567499
validation accuracy: 0.8077511473737888


# TensorFlowで分類する

In [7]:
# 必要なデータを再度抽出
X_train = train_df["terms"].values
X_test  = test_df["terms"].values

# 1から5までの数値になっているのでそれを0から4に変換する
y_train = train_df["label"].astype(float) - 1
y_test  = test_df["label"].astype(float) - 1

num_labels = len(y_train.unique())

In [8]:
# Token化するためのレイヤーを定義
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
vocab_size = 20000
sequence_length = 256

vectorize_layer = TextVectorization(
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length)

vectorize_layer.adapt(X_train)

# 1. Dense (+Dropout) でモデル構築

In [11]:
embedding_dim=256

model = Sequential([
  vectorize_layer,
  Embedding(vocab_size, embedding_dim, name="embedding"), # output shape=(17647, 256, 256): vocab_size, sequence_length, embedding_dim
  GlobalAveragePooling1D(),
  Dropout(0.2),
  Dense(256, activation='relu'),
  Dropout(0.2),
  Dense(64, activation='relu'),
  Dense(num_labels, activation="softmax")
])
model.compile(optimizer='adam',
              loss=SparseCategoricalCrossentropy(from_logits=True),
              metrics=['acc'])

callbacks = [EarlyStopping(
    monitor='val_loss', patience=5)]

dense_history = model.fit(
    X_train, 
    y_train,
    validation_data=(X_test, y_test),
    epochs=20,
    callbacks=callbacks)

min_index = np.argmin(dense_history.history["val_loss"])
print("min val_loss:", dense_history.history["val_loss"][min_index], "val_acc:", dense_history.history["val_acc"][min_index])

Epoch 1/20
552/552 [==============================] - 15s 26ms/step - loss: 1.3464 - acc: 0.4624 - val_loss: 1.0028 - val_acc: 0.5747
Epoch 2/20
552/552 [==============================] - 14s 25ms/step - loss: 0.9074 - acc: 0.6293 - val_loss: 0.8649 - val_acc: 0.6609
Epoch 3/20
552/552 [==============================] - 14s 25ms/step - loss: 0.6855 - acc: 0.7373 - val_loss: 0.8115 - val_acc: 0.6996
Epoch 4/20
552/552 [==============================] - 14s 25ms/step - loss: 0.5352 - acc: 0.8016 - val_loss: 0.8100 - val_acc: 0.7190
Epoch 5/20
552/552 [==============================] - 14s 25ms/step - loss: 0.4117 - acc: 0.8539 - val_loss: 0.7799 - val_acc: 0.7542
Epoch 6/20
552/552 [==============================] - 14s 25ms/step - loss: 0.3389 - acc: 0.8777 - val_loss: 0.8547 - val_acc: 0.7430
Epoch 7/20
552/552 [==============================] - 14s 25ms/step - loss: 0.2651 - acc: 0.9078 - val_loss: 0.9106 - val_acc: 0.7328
Epoch 8/20
552/552 [==============================] - 14s 25ms

# 2. LSTM

In [16]:
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, Dropout, Bidirectional, LSTM, BatchNormalization
model_lstm = Sequential([
    vectorize_layer,
    Embedding(vocab_size, embedding_dim, mask_zero=True), # output shape=(17647, 256, 256): vocab_size, sequence_length, embedding_dim
    Bidirectional(LSTM(256,  return_sequences=True)),
    Bidirectional(LSTM(128)),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(num_labels, activation="softmax")
])
model_lstm.compile(optimizer='adam',
              loss=SparseCategoricalCrossentropy(from_logits=True),
              metrics=['acc'])
lstm_history = model_lstm.fit(
    X_train, 
    y_train,
    validation_data=(X_test, y_test),
    epochs=20,
    callbacks=callbacks)

min_index = np.argmin(lstm_history.history["val_loss"])
print("min val_loss:", lstm_history.history["val_loss"][min_index], "val_acc:", lstm_history.history["val_acc"][min_index])

Epoch 1/20
552/552 [==============================] - 74s 116ms/step - loss: 1.2721 - acc: 0.4847 - val_loss: 1.0107 - val_acc: 0.5752
Epoch 2/20
552/552 [==============================] - 61s 111ms/step - loss: 0.8977 - acc: 0.6286 - val_loss: 0.8911 - val_acc: 0.6384
Epoch 3/20
552/552 [==============================] - 61s 111ms/step - loss: 0.6761 - acc: 0.7402 - val_loss: 0.8487 - val_acc: 0.6747
Epoch 4/20
552/552 [==============================] - 61s 111ms/step - loss: 0.4776 - acc: 0.8243 - val_loss: 0.8076 - val_acc: 0.7257
Epoch 5/20
552/552 [==============================] - 61s 111ms/step - loss: 0.3466 - acc: 0.8776 - val_loss: 0.9079 - val_acc: 0.7491
Epoch 6/20
552/552 [==============================] - 61s 111ms/step - loss: 0.2255 - acc: 0.9254 - val_loss: 0.9884 - val_acc: 0.7430
Epoch 7/20
552/552 [==============================] - 61s 111ms/step - loss: 0.1752 - acc: 0.9422 - val_loss: 0.9742 - val_acc: 0.7828
Epoch 8/20
552/552 [==============================] - 6

# 3. TD-IDF + Dense

In [25]:
# TF-IDFでToken化するためのレイヤーを定義
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
vocab_size = 20000

tfidf_vectorize_layer = TextVectorization(
    max_tokens=vocab_size,
    output_mode='tf-idf',
    ngrams=(1, 2)
)

tfidf_vectorize_layer.adapt(train_df["terms"].values)

In [33]:
model_tfidf = Sequential([
    tfidf_vectorize_layer,
    Dense(256, activation='relu'),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(num_labels, activation="softmax")
])
model_tfidf.compile(optimizer='adam',
              loss=SparseCategoricalCrossentropy(from_logits=True),
              metrics=['acc'])
tfidf_history = model_tfidf.fit(
    X_train,
    y_train,
    validation_data=(X_test, y_test),
    epochs=10,
    callbacks=callbacks)

min_index = np.argmin(tfidf_history.history["val_loss"])
print("min val_loss:", tfidf_history.history["val_loss"][min_index], "val_acc:", tfidf_history.history["val_acc"][min_index])

Epoch 1/10
552/552 [==============================] - 9s 16ms/step - loss: 1.1918 - acc: 0.5405 - val_loss: 0.7081 - val_acc: 0.7307
Epoch 2/10
552/552 [==============================] - 9s 15ms/step - loss: 0.4154 - acc: 0.8549 - val_loss: 0.6192 - val_acc: 0.7904
Epoch 3/10
552/552 [==============================] - 8s 15ms/step - loss: 0.1679 - acc: 0.9505 - val_loss: 0.6515 - val_acc: 0.8072
Epoch 4/10
552/552 [==============================] - 8s 15ms/step - loss: 0.1165 - acc: 0.9744 - val_loss: 0.8522 - val_acc: 0.8123
Epoch 5/10
552/552 [==============================] - 9s 15ms/step - loss: 0.0686 - acc: 0.9832 - val_loss: 0.8639 - val_acc: 0.8032
Epoch 6/10
552/552 [==============================] - 8s 15ms/step - loss: 0.0461 - acc: 0.9876 - val_loss: 0.8379 - val_acc: 0.8134
Epoch 7/10
552/552 [==============================] - 9s 15ms/step - loss: 0.0659 - acc: 0.9834 - val_loss: 0.9092 - val_acc: 0.8083
min val_loss: 0.6191643476486206 val_acc: 0.7904130816459656


# 参考. nnlm-ja-dim128 + Dense

In [60]:
# 必要なデータを再度抽出
X_train = train_df["terms"].values
X_test  = test_df["terms"].values
y_train = train_df["label"].astype(float) - 1
y_test  = test_df["label"].astype(float) - 1
num_labels = len(y_train.unique())

train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1961/1961 [00:04<00:00, 414.01it/s]


In [85]:
import tensorflow_hub as hub
import tensorflow as tf
hub_layer = hub.KerasLayer("https://tfhub.dev/google/nnlm-ja-dim128-with-normalization/2",
                           input_shape=[], dtype=tf.string)
model = Sequential()
model.add(hub_layer)
model.add(Dense(16, activation='relu'))
model.add(Dense(num_labels, activation='softmax'))

model.summary()

model.compile(optimizer='adam',
              loss=SparseCategoricalCrossentropy(from_logits=True),
              metrics=['acc'])

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_6 (KerasLayer)   (None, 128)               117568128 
_________________________________________________________________
dense_6 (Dense)              (None, 16)                2064      
_________________________________________________________________
dense_7 (Dense)              (None, 5)                 85        
Total params: 117,570,277
Trainable params: 2,149
Non-trainable params: 117,568,128
_________________________________________________________________


In [86]:
callbacks = [EarlyStopping(
    monitor='val_loss', patience=5)]

history = model.fit(train_dataset.shuffle(10000).batch(512),
    validation_data=test_dataset.batch(512),
    epochs=200,
    callbacks=callbacks)
min_index = np.argmin(history.history["val_loss"])
print("min val_loss:", history.history["val_loss"][min_index], "val_acc:", history.history["val_acc"][min_index])

Epoch 1/200
35/35 [==============================] - 1s 19ms/step - loss: 1.5659 - acc: 0.2739 - val_loss: 1.4369 - val_acc: 0.4294
Epoch 2/200
35/35 [==============================] - 1s 16ms/step - loss: 1.4162 - acc: 0.4337 - val_loss: 1.4053 - val_acc: 0.4314
Epoch 3/200
35/35 [==============================] - 1s 16ms/step - loss: 1.3797 - acc: 0.4407 - val_loss: 1.3656 - val_acc: 0.4544
Epoch 4/200
35/35 [==============================] - 1s 16ms/step - loss: 1.3416 - acc: 0.4657 - val_loss: 1.3214 - val_acc: 0.4686
Epoch 5/200
35/35 [==============================] - 1s 16ms/step - loss: 1.2891 - acc: 0.4951 - val_loss: 1.2803 - val_acc: 0.5105
Epoch 6/200
35/35 [==============================] - 1s 16ms/step - loss: 1.2677 - acc: 0.5044 - val_loss: 1.2466 - val_acc: 0.5161
Epoch 7/200
35/35 [==============================] - 1s 16ms/step - loss: 1.2212 - acc: 0.5181 - val_loss: 1.2197 - val_acc: 0.5201
Epoch 8/200
35/35 [==============================] - 1s 16ms/step - loss: 1.